<a href="https://colab.research.google.com/github/yashaswip/Assignment-1/blob/main/M3_Part_I_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Assignment 3 Naïve Bayes and Sentiment Classification and Logistic Regression
Instructions
* Read the following Chapter 4: Naive Bayes and Sentiment Classification. Speech and Language Processing. Daniel Jurafsky & James H. Martin. Copyright © 2021. All rights reserved. Draft of September 21, 2021. I have tried to pull out relevant notes for you below, but it is encouraged that you read each chapter provided.
* Read the following Chapter 5: Logistic Regression. Speech and Language Processing. Daniel Jurafsky & James H. Martin. Copyright © 2021. All rights reserved. Draft of September 21, 2021. I have tried to pull out relevant notes for you below, but it is encouraged that you read each chapter provided.

Summary
Classification is one of the most important tasks of NLP and in machine learning. In NLP it often means the task of text categorization for both sentiment analysis, spam detection, and topic modeling. Naïve Bayes is often one of the first classification algorithms defined in NLP.  The intuition behind a classifier is lies at the underlying probability inferred by the Bayesian Inference, which uses Baye’s rule and conditional probabilities.

Here’s a reminder on Baye’s Rule:
P(y)=P(x)P(x)/(P(y))

We are saying “what is the probability of x given y”. Naïve Bayes is a generative model because there is an input that helps the model determine what the output could be. Said differently, “to train a generative model we first collect a large amount of data in some domain (e.g., think millions of images, sentences, or sounds, etc.) and then train a model to generate data like it.” [6]

So in the case of Naïve Bayes, we say given some word, what should be the class of the current word we are assessing? Contrastingly, discriminative models such as logistic regression, learn from features provided to the algorithm and then determine or predict what the class is. [7]


With Naïve Bayes, the assumption is that the probabilities are independent. We often call the Naïve Bayes classifier the bag-of-words approach. That’s because we are essentially throwing in the collection of words into a ‘bag’, selecting a word at random, and then calculating their frequency to use in the Bayesian Inference. Thus, context – the position of words -- is ignored and despite this, it turns out that the Naïve Bayes approach can be accurate and effective at determining whether an email is spam for example.

Back to bag of words. With bag of words, we assume that the position of the words are not relevant -- that dependency or context in the word phrase or sentence doesn’t matter. Relatedly, the naive Bayes assumption implies that the conditional probabilities are independent -- a rather strange assumption to make for words in a sentence! The equation for the naive Bayes classifier is outlined below:

You can use Naive Bayes by creating an index of words and walking through every word position in a test or corpus.


It can be useful to be able to classify new "test" documents using already classified "training" documents.  A common example is using a corpus of labeled spam and ham (non-spam) e-mails to predict whether or not a new document is spam.

For this Assignment, you can start with a spam/ham dataset, then predict the class of new documents (either withheld from the training dataset or from another source such as your own spam folder).   One example corpus:   https://spamassassin.apache.org/old/publiccorpus/

You may work alone or in a group on this project.  You're welcome to use any tools or approach that you like.  Due before our next meetup. Starter code provided below.

Test example is provided at the end.

Libraries you may wish to use

In [6]:
import pandas as pd
import numpy as np
from os import makedirs, path, remove, rename, rmdir
from tarfile import open as open_tar
from shutil import rmtree
from urllib import request, parse
from glob import glob
from re import sub
from email import message_from_file
from sklearn.model_selection import train_test_split
from collections import defaultdict
from functools import partial
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score)
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import os
import email
from email.policy import default

download corpus using the following functions

Note: you may need to mount your drive on google then run this location. See previous exercises.

In [19]:
# Function to download and prepare the dataset
def download_corpus(dataset_dir: str = 'data'):
    base_url = 'https://spamassassin.apache.org'
    corpus_path = 'old/publiccorpus'
    files = {
        '20021010_easy_ham.tar.bz2': 'ham',
        '20021010_hard_ham.tar.bz2': 'ham',
        '20021010_spam.tar.bz2': 'spam',
        '20030228_easy_ham.tar.bz2': 'ham',
        '20030228_easy_ham_2.tar.bz2': 'ham',
        '20030228_hard_ham.tar.bz2': 'ham',
        '20030228_spam.tar.bz2': 'spam',
        '20030228_spam_2.tar.bz2': 'spam',
        '20050311_spam_2.tar.bz2': 'spam' }

    # Create the folders: downloads, ham and spam
    downloads_dir = os.path.join(dataset_dir, 'downloads')
    ham_dir = os.path.join(dataset_dir, 'ham')
    spam_dir = os.path.join(dataset_dir, 'spam')

    os.makedirs(downloads_dir, exist_ok=True)
    os.makedirs(ham_dir, exist_ok=True)
    os.makedirs(spam_dir, exist_ok=True)

    for file, spam_or_ham in files.items():
        # Download files from URL of each specific .bz2 file
        url = parse.urljoin(base_url, f'{corpus_path}/{file}')
        tar_filename = os.path.join(downloads_dir, file)
        request.urlretrieve(url, tar_filename)

        # List emails in the compressed .bz2 file
        emails = []
        with open_tar(tar_filename) as tar:
            tar.extractall(path=downloads_dir)
            for tarinfo in tar:
                if len(tarinfo.name.split('/')) > 1:
                    emails.append(tarinfo.name)

        # Move emails to ham or spam directory
        for email in emails:
            directory, filename = email.split('/')
            directory = os.path.join(downloads_dir, directory)

            if not os.path.exists(os.path.join(dataset_dir, spam_or_ham, filename)):
                os.rename(os.path.join(directory, filename),
                       os.path.join(dataset_dir, spam_or_ham, filename))

        rmtree(directory)

# Download the corpus
download_corpus()

#How many e-mails are classified in our dataset as either Spam or not Spam?


In [20]:
# Function to load emails from a directory
def load_emails_from_directory(directory):
    emails = []
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r', errors='ignore') as file:
                try:
                    msg = email.message_from_file(file, policy=default)
                    for part in msg.walk():
                        if part.get_content_type() == 'text/plain':
                            emails.append(part.get_payload())
                except Exception as e:
                    print(f"Error reading file {file_path}: {e}")
    return emails

# Load the emails
ham_emails = load_emails_from_directory(ham_dir)
spam_emails = load_emails_from_directory(spam_dir)

print(f'Loaded {len(ham_emails)} ham emails and {len(spam_emails)} spam emails')


Loaded 6743 ham emails and 1341 spam emails


Provide your classifier below

In [22]:
##### START CODE ####
# Combine and Label the Emails
all_emails = ham_emails + spam_emails
labels = [0] * len(ham_emails) + [1] * len(spam_emails)  # 0 for ham, 1 for spam

X_train, X_test, y_train, y_test = train_test_split(all_emails, labels, test_size=0.2, random_state=42)

# Vectorize the Emails
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

# Train the Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train_counts, y_train)

##### END CODE ####


MultinomialNB()

The following email is a test email. You can take this and test your classifier to see if it predicts spam or not.



In [24]:
spam_email = """
Subject: Get Rich Quick!

Dear Friend,

Congratulations! You've been selected to participate in an exclusive opportunity to make thousands of dollars from the comfort of your own home. Our revolutionary system guarantees quick and easy cash with minimal effort.

No more struggling to pay bills or worrying about financial security. With our proven method, you can start earning massive amounts of money in no time.

Here's what some of our satisfied customers have to say:
- "I was skeptical at first, but I'm now living my dream life thanks to this incredible system!" - John S.
- "I never thought making money online could be this simple. It's changed my life!" - Sarah L.

Don't miss out on this limited-time offer. Act now to secure your spot and start enjoying a life of financial freedom.

Click the link below to get started:
www.getrichquick.com

Remember, this opportunity is exclusive and won't last long. Take control of your financial future today!

Best regards,
The Get Rich Quick Team
"""
 #Transform the sample email to the same format as our training data
sample_email_counts = vectorizer.transform([spam_email])
sample_pred = clf.predict(sample_email_counts)

print('Spam' if sample_pred[0] == 1 else 'Not Spam')


Spam
